# Content and Objective

+ Show validity of Goertzel's algorithm

# Importing and Plotting Options

In [1]:
import numpy as np

from scipy import signal
from time import time 

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# plotting options 
font = {'size'   : 20}
plt.rc('font', **font)
plt.rc('text', usetex=0)

matplotlib.rc('figure', figsize=(18, 6) )

# Parameters

In [3]:
# define parameters
    
# dft length
N = 64

n = np.arange( 0, N )
k = np.arange( - N//2, N//2 )

# Define signal

In [4]:
switch = 3

if switch == 1:
    x = np.random.rand( N )
elif switch == 2:
    x = np.random.randn( N )
elif switch == 3:
    x = np.exp( 1j * 2 * np.pi * n * (N//10) / N )


# Plotting signal and DFT

In [5]:
plt.figure(1)

plt.subplot(121)
plt.stem(n, x.real,label='$Re\{ x[n]\}$')
plt.grid(True);  plt.xlabel('$n$'); #plt.ylabel('$Re\{ x[n]\}$')
plt.axis(xmin=0, xmax=N+1, ymin=-1.1, ymax=1.1)
plt.legend(loc='upper right')

X = np.fft.fft( x ) 

plt.subplot(122)
plt.stem( k, np.abs( np.fft.fftshift(X) )**2,label='$|X[k]|^2$')
plt.grid(True);  plt.xlabel('$k$'); #plt.ylabel('$|X[k]}|^2$')
#plt.axis(xmin=0, xmax=N+1) #, ymin=-1.1, ymax=1.1)
plt.legend(loc='upper right')

# Show that Goertzel actually works and is way faster

+ We will increase order of FFT significantly to show the effects
+ *NOTE:* Since the numpy implementation of the FFT is so efficient, the speed-up, unfortunately, not be observed.
+ Nevertheless, we will be able to verify that the Goertzel algorithm actually determines the according FFT value

In [6]:
# define parameters
    
# dft length
N = 2**3

n = np.arange( 0, N )
k = np.arange( - N//2, N//2 )

# Index at which FFT should be determined
k_0 = N//5

In [7]:
switch = 2

if switch == 1:
    x = np.random.rand( N )
elif switch == 2:
    x = np.random.randn( N )
elif switch == 3:
    x = np.exp( 1j * 2 * np.pi * n * (N//10) / N )

In [8]:
# Show duration required for FFT
start = time()
X = np.fft.fft( x )
X_0_fft = X[ k_0 ]
elapsed_fft = time() - start

print('Time for FFT: \t\t', elapsed_fft )
print('Value X[ k_0 ]: \t', X_0_fft )

Time for FFT: 		 0.0
Value X[ k_0 ]: 	 (-3.1220642814020887-1.5081113529271049j)


In [9]:
# Goertzel algorithm

def Goertzel_iterative( x, k_0 ):
    '''
    Performs Goertzel's algorithm on the input sequence using iterations

    IN: signal x
        index k_0 at which the FFT has to be found
    '''

    s = np.zeros( N+3 )
    x_padded = np.concatenate( ( [ 0, 0], x , [ 0 ] ) )

    omega = 2 * np.pi * k_0 / N

    for i in range( 2, N+3  ):
        s[ i ] = x_padded[ i ] + 2 * np.cos( omega ) * s[i-1] - s[i-2] 

    y =	s[ -1 ] - np.exp( -1j* omega ) * s[ -2 ]

    return y

In [10]:
# Show duration required for Goertzel
start = time()
X_0_goertzel_iterative = Goertzel_iterative( x, k_0 )
elapsed_goertzel_iterative = time() - start

print('Time for Goertzel: \t', elapsed_goertzel_iterative )
print('Value X[ k_0 ]: \t', X_0_goertzel_iterative )

Time for Goertzel: 	 0.0010151863098144531
Value X[ k_0 ]: 	 (-3.1220642814020882-1.5081113529271049j)


In [11]:
# Goertzel algorithm as filter

def Goertzel_filter( x, k_0 ):
    '''
    Performs Goertzel's algorithm on the input sequence using filtering

    IN: signal x
        index k_0 at which the FFT has to be found
    '''

    omega = 2 * np.pi * k_0 / N
    x_padded = np.concatenate( ( x , [0] ) ) 

    # Define the system filter: [1, -coeff, 1] (difference equation)
    b = np.array( [ 1.0, - np.exp( -1j * omega) ] )  # Input gain (x[n])
    a = np.array( [ 1.0, - 2 * np.cos( omega ), 1.0 ] )  # Denominator coefficients

    # Apply the filter using lfilter (vectorized recurrence)
    s = signal.lfilter(b, a, x_padded)

    y =	s[-1] #- np.exp( -1j* omega ) * s[-2]

    return y

In [12]:
# Show duration required for Goertzel
start = time()
X_0_goertzel_filter = Goertzel_filter( x, k_0 )
elapsed_goertzel_filter = time() - start

print('Time for Goertzel w. filter: \t', elapsed_goertzel_filter )
print('Value X[ k_0 ]: \t\t', X_0_goertzel_filter )

Time for Goertzel w. filter: 	 0.0
Value X[ k_0 ]: 		 (-3.1220642814020882-1.5081113529271044j)


# Compare Performance for different input length

In [ ]:
if 0:
    N_exp = np.arange( 5, 12 )
    N_fft = 2**N_exp
else:
    N_fft = np.arange( 2**5, 2**16, 1000 )


N_trials = 1000

time_elapsed = np.zeros( (3, len(N_fft)) )

for ind_t, val_t in enumerate( N_fft ):

    start = time()
    for _n in range(N_trials):
        x = np.random.randn( val_t  )
        X_fft = np.fft.fft( x )
        X_0_fft = X_fft[ k_0 ]
    time_elapsed[ 0, ind_t ] = time() - start

    start = time()
    for _n in range(N_trials):
        x = np.random.randn( val_t  )
        X_0_goertzel_iterative = Goertzel_iterative( x, k_0 )
    time_elapsed[ 1, ind_t ] = time() - start


    start = time()
    for _n in range(N_trials):
        x = np.random.randn( val_t  )
        X_0_goertzel_filter = Goertzel_filter( x, k_0 )
    time_elapsed[ 2, ind_t ] = time() - start


In [ ]:
plt.figure()

plt.plot( N_fft, time_elapsed[ 0, :], label='FFT' )
plt.plot( N_fft, time_elapsed[ 1, :], label='Goertzel iterative' )
plt.plot( N_fft, time_elapsed[ 2, :], label='Goertzel filter' )
plt.yscale('log')
plt.xlabel()
plt.grid(1)
plt.legend()
plt.show()